In [ ]:
import bibtexparser
import requests
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
tqdm.pandas()

In [ ]:
def read_bib_file(path):
    with open(path, 'r', encoding='utf-8') as fichier:
        contenu_bib = fichier.read()
    parser = bibtexparser.bparser.BibTexParser(common_strings=True)
    bib_database = bibtexparser.loads(contenu_bib, parser=parser)
    return bib_database

In [ ]:
wgi=[]
dataframes_i={}
for i in ['01','02','03','04','05','06','07','08','09','10','11','12']:
    df=pd.DataFrame(read_bib_file(f"../IPCC_bibliography/AR6/WG1/IPCC_AR6_WGI_References_Chapter{i}.bib").entries)
    df['wg']='wg1'
    df['chap']=f"wg1_chap_{i}"
    wgi.append(len(df.doi.dropna())/len(df))
    df_name = f'df_i_{i}' 
    dataframes_i[df_name] = df
print(wgi)

In [ ]:
wgiii=[]
dataframes_iii={}
for i in ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17']:
    print(i)
    df=pd.DataFrame(read_bib_file(f"../IPCC_bibliography/AR6/WG3/IPCC_AR6_WGIII_References_Chapter{i}.bib").entries)
    df['wg']='wg3'
    df['chap']=f"wg3_chap_{i}"
    wgiii.append(len(df.doi.dropna())/len(df))
    df_name = f'df_iii_{i}' 
    dataframes_iii[df_name] = df
print(wgiii)

Get doi from glutton

In [ ]:
def get_doi(row):
    title=row.title
    author=str(row.author).split(',')[0]
    url=f"https://cloud.science-miner.com/glutton/service/lookup?atitle={title}&firstAuthor={author}"
    response = requests.get(url)
    data = response.json()
    return data.get('DOI')

In [ ]:
wgii=[]
wgii_update=[]
dataframes_ii = {}
for i in ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18']:
    df=pd.DataFrame(read_bib_file(f"../IPCC_bibliography/AR6/WG2/IPCC_AR6_WGII_References_Chapter{i}.bib").entries)
    wgii.append(len(df.doi.dropna())/len(df))
    df.loc[pd.isna(df.doi),'doi']=df.loc[pd.isna(df.doi),:].progress_apply(get_doi, axis=1)
    wgii_update.append(len(df.doi.dropna())/len(df))
    df['wg']='wg2'
    df['chap']=f"wg2_chap_{i}"
    df_name = f'df_ii{i}' 
    dataframes_ii[df_name] = df
print(wgii)
print(wgii_update)

In [ ]:
wgii_cross=[]
wgii_cross_update=[]
dataframes_cross_ii = {}

for i in range(1,8):
    df=pd.DataFrame(read_bib_file(f"../IPCC_bibliography/AR6/WG2_CROSS/IPCC_AR6_WGII_References_CCP{i}.bib").entries)
    wgii_cross.append(len(df.doi.dropna())/len(df))
    df.loc[pd.isna(df.doi),'doi']=df.loc[pd.isna(df.doi),:].progress_apply(get_doi, axis=1)
    wgii_cross_update.append(len(df.doi.dropna())/len(df))
    df['wg']='wg2_cross'
    df['chap']=f"wg2_cross_chap_{i}"
    df_name = f'df_cross_ii_{i}' 
    dataframes_cross_ii[df_name] = df
print(wgii_cross)
print(wgii_cross_update)

Clean DOI

In [ ]:
path='../IPCC_bibliography/AR6/structured_data/'

In [ ]:
#data for each wg
df_wgi = pd.concat(list(dataframes_i.values()), ignore_index=True)
df_wgii = pd.concat(list(dataframes_ii.values()), ignore_index=True)
df_cross_wgii = pd.concat(list(dataframes_cross_ii.values()), ignore_index=True)
df_wgiii = pd.concat(list(dataframes_iii.values()), ignore_index=True)

In [ ]:
df_wgi.to_json(path+'data_wg1.jsonl', orient='records', lines=True)
df_wgii.to_json(path+'data_wg2.jsonl', orient='records', lines=True)
df_cross_wgii.to_json(path+'data_cross_wg2.jsonl', orient='records', lines=True)
df_wgiii.to_json(path+'data_wg3.jsonl', orient='records', lines=True) 